In [1]:
import os
import pandas as pd
import numpy as np
import glob

In [26]:
sub = 'sub-12'
ses = 'ses-02'
battery = 'camcan'
task = 'VSTMC'

main_dir = '/home/sshankar/ibc/analysis_pipeline/ibc_main/neurospin_data/info/'
datadir = os.path.join(main_dir, sub, battery, ses)
os.chdir(datadir)
csv_files = sorted(glob.glob('*' + task + '*.txt'))
tsv_files = []
# for i in range(len(csv_files)):
i = 0
tsv_files.append(task + '_' + sub + '_run-0' + str(i+1) + '.tsv')

In [27]:
# In the log file run start is the file line.
# To extract it read in the first line by itself and then 
# read the rest of the file as a DataFrame.
line1 = pd.read_csv(os.path.join(datadir, csv_files[i]), sep=':', header=None, nrows=1)
run_start = line1[1][0]

In [28]:
run_start

125.4668

In [29]:
# Read in the rest of the file
file_df = pd.read_csv(os.path.join(datadir, csv_files[i]), header=1, sep=',', skipinitialspace=True)

In [30]:
# Extract the stimulus load (how many motion directions to remember: 1, 2 or 3)
loads = file_df['Load']

In [31]:
# Extract the onsets of stimulus, probe and response
stim_onsets = file_df['Stim1_Onset']
stim_offsets = file_df['Delay_Onset']
stim_durations = stim_offsets - stim_onsets
stim_types = np.repeat('stim_load', len(stim_onsets))
stims = [i+str(j) for i, j in zip(stim_types, loads.values)]

resp_period_onsets = file_df['Probe_Onset']
resp_period_offsets = file_df['ResponseTime']
resp_period_durations = resp_period_offsets - resp_period_onsets
resp_types = np.repeat('resp_load', len(stim_onsets))
resps = [i+str(j) for i, j in zip(resp_types, loads.values)]

In [32]:
# Create onsets, durations and trial_types vectors that are a zipped merge 
# of the respective stim and resp vectors
trial_onsets = (np.vstack((stim_onsets,resp_period_onsets))).flatten('F')
durations = (np.vstack((stim_durations,resp_period_durations))).flatten('F')
trial_types = (np.vstack((stims,resps))).flatten('F')

In [33]:
# Create a DataFrame to save to events file
df = pd.DataFrame({'onset':(trial_onsets-run_start), 'duration':durations, 'trial_type':trial_types})
df.to_csv(os.path.join(datadir, tsv_files[0]), sep='\t', float_format='%0.3f', index=False)